# Tutorial: K-means Clustering


# 0. Introduction to the Course
Welcome the tutorial for **k-means clustering**. The lesson that follows will serve as a non-exhaustive guide to the basics of k-means clustering, answering questions that you may have such as...
- What even *is* a cluster?
- What is the importance of clustering analysis?
- What makes k-means clustering different from other types of clustering algorithms?  

All that and more will be covered in this tutorial, and by the end of it we hope that you will have a solid background in regards to what cluster analysis is, how to perform k-means clustering by yourself, and how to interpret the resulting data, all of which will come in handy when you apply it to your *own* data analysis projects.  

The following is a brief outline of the structure of this tutorial, highlighting the main topics that we will cover:
1. Introduction to cluster analysis
    - Intro to feature vectors and data frames
    - Intro to plotting and transformation
2. Introduction to k-means clustering
    - Basic theory
    - Basic clustering and calculations
    - 'Within-cluster sum of squares' models
    - The k-means algorithm
        - Limitations
    - `sklearn` and the `iris` dataset
6. Concluding remarks


# 1. Introduction to Cluster Analysis
Let's start with the basics and define what we we're talking about when referring to a **cluster**, and why this topic is so important to data scientists.  

Broadly speaking, a **cluster** is a *collection of similar objects*.

We can call many different things cluster, at the same defining what quality it is that defines a particular cluster. For instance, let's take the following list of items:

```python
random_objects = ['eggplant','grape','banana','school bus','plum','fire hydrant']
```

How would we go about defining what clusters are in this list of objects?  
Some observations might strike out at you immediately, for instance:  
- `eggplant`, `grape`, and `plum` can form a cluster of things that are *purple* 
    - likewise `banana` and `school bus` form a cluster of things that are *yellow*
- `grape`, `banana`, and `plum` can form a cluster of things that are *fruit*
- `school bus` and `fire hydrant` can form of cluster of things that are *NOT edible*

This is the basic nature of clustering: we ask ourselves what *associations* exist in our data, and then make groups out of objects that associate with each other. As this example demonstrated, there can be *multiple* "right" answers, and indeed this highlights three main questions that clustering analysis seeks to solve:
1. How many clusters are in my data?
2. How do I determine what goes into each cluster?
3. Which clustering of my data is **"most correct"**?  

As we move forward you'll be able to get a better sense of how to answer these questions, and you will see how k-means clustering gives us a way to answer the last question, which may have seemed like the most unintuitive to answer of the bunch.

### Intro to Feature Vectors and Data Frames

Before we dive into k-means, let's up the difficulty one more step. In our last example, we used extremely basic data, and all of the **features** that we used to mentally cluster these objects were not explicitly written out - we were just going off of memory. In many data science applications, scientists use what is called a **feature vector** in order to compress the qualities of an object into an easy-to-use format. 

Let's work through an example with the first element of our `random_objects` list, `eggplant`.  

Eggplants, as I'm sure we know, are complex objects. Any single quality of an eggplant can be incorporated into an eggplant feature vector. Let's try constructing one now as a `list`. Fill in the features below with the indicated data type (don't worry about being exact).

```python
color =    #string 
taste =    #string 
type =    #string 
avg_price =    #int, dollars
avg_mass =    #int, pounds
edible =    #boolean
eggplant = [color, taste, type, avg_price, avg_mass, edible]
eggplant
```  

We now have a feature vector describing at least *some* of the attributes of an eggplant. The variables that we used in this example define the **feature space** of our analysis - it is the collection of features that we're actively looking at in our data. Note how if we instead used the following feature vector...

```python
eggplant = [color, taste, type, avg_price/avg_mass, edible]
```

... We would be in a different feature space, despite using the same variables to construct it.  

Below we've assembled a table combining the different objects in our `random_objects` list under an identical feature space. (Don't worry if your eggplant vector was different). If you are familiar with the R programming language, you will recognize this data structure as a **data frame** - an array-like object in which each column corresponds to a measurement, quantitative *or* qualitative, of a variable, and each row corresponds to a unique object. Alternatively, you can picture this data frame as a *row-wise collection of feature vectors*.

<table>
  <thead>
    <tr>
      <th></th>
      <th>color</th>
      <th>taste</th>
      <th>type</th>
      <th>avg_price</th>
      <th>avg_mass</th>
      <th>edible</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>eggplant</td>
      <td>purple</td>
      <td>bland</td>
      <td>food</td>
      <td>3</td>
      <td>1</td>
      <td>True</td>
    </tr>
    <tr>
      <td>grape</td>
      <td>purple</td>
      <td>sweet</td>
      <td>food</td>
      <td>5</td>
      <td>0.5</td>
      <td>True</td>
    </tr>
    <tr>
      <td>banana</td>
      <td>yellow</td>
      <td>sweet</td>
      <td>food</td>
      <td>4</td>
      <td>2</td>
      <td>True</td>
    </tr>
    <tr>
      <td>school bus</td>
      <td>yellow</td>
      <td>metallic</td>
      <td>utility</td>
      <td>5000</td>
      <td>1000</td>
      <td>False</td>
    </tr>
    <tr>
      <td>plum</td>
      <td>purple</td>
      <td>sweet</td>
      <td>food</td>
      <td>1</td>
      <td>0.75</td>
      <td>True</td>
    </tr>
    <tr>
      <td>fire hydrant</td>
      <td>red</td>
      <td>metallic</td>
      <td>utility</td>
      <td>500</td>
      <td>100</td>
      <td>False</td>
    </tr>
  </tbody>
</table>

In Python, data frames can easily be implemented using the **pandas** library. Colloquically, pandas is imported with the abreviation 'pd' like so:

```python
import pandas as pd
```

To see more information about the `pd.DataFrame` class, you can type `?pd.DataFrame` in your Python console, or visit [this url](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). As a starter exercise, let's recreate the table above into a pandas data frame by calling `pd.DataFrame`. Note from the help document or the website what arguments the call to `pd.DataFrame` takes:
- A **data array**
- A list of **sample indices** (or row names)
- A list of **column labels** (or column names)

From our sample dataset, we know that our `random_objects` list contains our 'sample indices', i.e. what we're looking at in our data. Likewise, we know that our feature space, the list of variables that we're looking at in our samples, is equivalent to our 'column labels' for this data frame. Finally, our actual 'data array' is what's left over - all the data about our samples. Let's work through creating our own data frame piece by piece.  

We've brought back our `random_objects` list from before. Now, referring to the table above, create a `column_labels` list containing strings representing the 6 variables that we're interested in.

```python
random_objects = ['eggplant','grape','banana','school bus','plum','fire hydrant']
column_labels = ['color','taste','type','avg_price','avg_mass','edible']
```

Good. Finally, let's define our data array and put everything together. The feature vectors from the table above have already been loaded for you below. Combine the feature vectors into a new list named `data`, then assemble the data frame object with `pd.DataFrame` using the arguments given above and save it as `df`. Call `df` at the end of code to check your work.

```python
eggplant = ['purple','bland','food', 3.5 , 1.2, True]
grape = ['purple', 'sweet', 'food', 5, 0.5, True]
banana = ['yellow', 'sweet', 'food', 4, 2, True]
school_bus = ['yellow', 'metallic', 'utility', 5000, 1000, False]
plum = ['purple', 'sweet', 'food', 1.30, 0.75, True]
fire_hydrant = ['red', 'metallic', 'utility', 500, 100, False]

data = [eggplant, grape, banana, school_bus, plum, fire_hydrant]

df = pd.DataFrame(data, random_objects, column_labels)
df
```

Great! Data frames are extremely useful structures for holding "mixed" datasets, i.e. those that have both qualitative and quantitative variables. They also have some unique properties compared to normal arrays. For instance, the indices, columns, and data that we used to build the data frame are now stored as attributes of the data frame. Give it a try below and see for yourself how the attributes line up with our inputs.

```python
df.index
df.columns
df.values
```

Additionally, a cool thing that we can do with data frames is to bring up rows of data not just by their index, but by their name. For instance, the code `df['color']` will return a *series* containing all the colors of our random objects with the name of the object next to the variable. *Slicing*, where we extract multiple lines of data from a data object, works just as easily; `df['color':'type']` will return the `color`, `type`, AND `taste` of each of our random objects, because `taste` is located between `color` and `type` in the column names. In the space below, please return the measurements of ALL variables but ONLY for eggplant, grape, and banana. (Hint: slicing can be used here too!)

```python
df['eggplant':'banana']
```

An extensive overview of what is possible with the pd.DataFrame class and the rest of the pandas library is available at [this site](http://pandas.pydata.org/pandas-docs/stable/) if you want to learn more. 

### Intro to Plotting and Transformation

To get back to clustering, let's do a very brief experiment that will briefly segue into two important parts of this tutorial - *plotting* and *data transformation* - that will be critical later on when we try to cluster bigger datasets. 

Looking at our data frame, we can see that there are different types of variables present. In fact, the attribute `df.dtypes` will tell you *exactly* what they are - a mix of objects, floats, and booleans. Clustering can be performed using any combination or weighting of the these variables. What we'll be doing in this experiment is testing the hypothesis that our dataset can be clustered by `avg_price` and `avg_mass`, the two features in our dataset that are strictly numbers, or `floats`. Thinking about it logically, we can intuit that objects that are heavier usually require more material to build, and thus will cost more to produce and distribute, so we predict a *positive correlation exists between `avg_price` and `avg_mass`*.  

The simplest way to test this is by plotting these data in a **scatter plot**. To do so we will need the **`matplotlib`** library, another commonly-used library that is colloquially imported as `plt`. If you are familiar with MATLAB, then a lot of the functions available in `matplotlib` should feel familiar to you. We will import the library as below, and then take a closer look at the `plt.scatter` class.

```python
import matplotlib as plt
?plt.scatter
```

The `plt.scatter` function is very easy to use - it simply takes an array-like object of `x` values, and a corresponding array-like object of `y` values. Let's try it out really quick. Recall how we used data frame indexing to grab columns that we want, and in the space below make a scatter plot of `avg_price` over `avg_mass` from our `df` data frame.

```python
## These two lines simply set up the graphical area with a 'figure' object and a 'plot' object 
## Plotting will be done in the 'plot' object
## Plot axis labels have been setup for your convenience
fig = plt.figure()
plot = fig.subplot(111)
plot.set_xlabel = ('avg_mass')
plot.set_ylabel = ('avg_price')

## Call the 'scatter' function on the `plot' object with the datasets discussed above  
plot.scatter(df['avg_mass'],df['avg_price'])
```
![bad plot](http://imgur.com/pfsUvO9)
Good, you've made the plot, though it seems like there's a lot of room for improvement. For one, we don't know which points correspond to which random object. We can also see that one point is extremely far from the rest, and it's skewing the axes. Let's try to fix these issues step-by-step.  

First, let's quickly look at our `df['avg_mass']` data. It's obvious that the two objects `school_bus` and `fire_hydrant` are outliers because of their large weight. You'll see a similar problem in `df['avg_price']`. To circumvent this type of issue, data scientists commonly ***transform*** their data so that all data points can be neatly plotted on the same axis without the type of 'stretching' that we saw in our plotting attempt. A common way to do this is by taking the *logarithm* of all values in a dataset, or equivalently, plotting the data on a *logarithmic axis*. What faster way to see how this works than to plot it!  In this next exercise, we will use the `np.log10` function from the numpy library to plot `avg_price` over the log-transformed `avg_mass`. 

```python
## As before, graphing parameters have been set for you
## Our 'figure' will have two plots, 'plot1' and 'plot2', that will be plotted on the same y-axis (sharey=True) 
fig, (plot1, plot2) = plt.subplots(1, 2, sharey=True)
plot1.set_xlabel('avg_mass')
plot1.set_ylabel('avg_price')
plot2.set_xlabel('log(avg_mass)')

## Our previous plot
plot1.scatter(df['avg_mass'],df['avg_price'])

## Insert the log-transformed plot here
plot2.scatter(np.log10(df['avg_mass']),df['avg_price'])
```
![better plot](http://imgur.com/wihzklD)
Well done. Note how we now have some degree of separation between the four points at the bottom. However, our graph is still skewed by the high `avg_price` of the two `school_bus` and `fire_hydrant` data points. Let us log-transform the `avg_price` column as well and plot it against our log-transformed `avg_mass` data.

```python
## 'plt.subplots(2, 1, sharex=True)' here means that we want our two subplots to be plotted side-by-side, row-wise
fig, (plo1, plot2) = plt.subplots(2, 1, sharex=True)
plot1.set_ylabel('avg_price')
plot2.set_xlabel('log(avg_mass)')
plot2.set_ylabel('log(avg_price)')
plot1.scatter(np.log10(new_data['avg_mass']), new_data['avg_price'])
plot2.scatter(np.log10(new_data['avg_mass']), np.log10(new_data['avg_price']))
```
![even better plot](http://imgur.com/cmZxBlE)
Good. You can see now in the bottom plot that our data looks much more presentable than before, and though two points seem like outliers compared to the cluster near the origin, the points are overall more spread apart and easier to distinguish. The only thing that remains is adding labels to our data points. Recall that our `random_objects` list still contains our list of objects that we're looking at. To add labels to our graph, we will use the `random_objects` list in junction with our data to create a *zip* object with the `zip()` function. The `zip()` function, very briefly, combines two array-like objects into a single array. For instance, in the following code:
```python
A = [1,2,3]
B = [4,5,6]
C = list(zip(A,B))
```
`C` will return the list `[(1,4), (2,5), (3,6)]`. This is very handy when plotting as it allows us to combine combine data sets and data labels without having to iterate over each one individually. Let's explore what happens when we `zip` our `random_objects` object with the data in our `avg_mass` and `avg_price` columns.  

First, let's convert the data into an array using numpy's `np.array()` function. Let's not forgot that we almost want to log10-transform the data using `np.log10()`.
```python
mass_price = np.log10(np.array(new_data[['avg_mass','avg_price']]))
```
Then let's `zip` the `random_objects` list with the `mass_price` array we just made, convert it to a list named `zip_data`, and print the first element. Do this by yourself in the space below.
```python
zip_data = list(zip(random_objects, mass_price)
print(zip_data[0])
```
Great! See how the `zip_data` object has condensed our data into 'random_object, mass_price' pairs? Let's use this new structure in order to label our last scatter plot from above. For this we'll need to iterate over the `zip_data` object, and use the `plt.annotate()` function from the matplotlib library. For our purposes, we'll use the `plt.annotate()` function with the following arguments:
```python
plt.annotate('text', xy, fontsize)
```
Where `text` is the label we want to assign to a point, `xy` are the coordinates where we want that label to go, and `fontsize` is self-explanatory. Let's put this all together below. On your own, design a `for` loop that iterates over the `zip_data` object we created above, and annotates each data point. We'll start you off with the plot specifications as before.
```python
fig, plot = plt.subplots(1, 1)
plot.scatter(np.log10(new_data['avg_mass']), np.log10(new_data['avg_price']))
plot.set_xlabel('log(avg_mass)')
plot.set_ylabel('log(avg_price)')

## start your 'for' loop here, using a fontsize of 10
for object, coord in zip_data:
    plot.annotate(object, coord, fontsize=10)
```
Good job. Just for convenience, let's rotate the labels a bit so that they're less cluttered. We can do this simply by adding a `rotation` argument, an `int` of how many degrees to rotate the labels, to our `plt.annotate` code, like so:
```python
for object, coord in zip_data:
    plot.annotate(object, coord, fontsize=10, rotation=-45)
```

![random objects plot](http://i.imgur.com/Z9RjxfI.png)

Great! Now we've added some 'substance' to our initial clustering assumptions - more than just randomly recalling the color or category that our random objects fall into from memory, we've shown how we can actually use quantitative observations from our random objects to visibily demonstrate how some objects from this group are more similar to each other. It's clear from our plot that `banana`, `grape`, `plum`, and `eggplant` - the four food items in our analysis - are in a separate *cluster* than `school_bus` and `fire_hydrant`, when we look at our data relative to the log-transformed `avg_mass` and `avg_price` features.  

Let's think about this last plot that we produced. Sure, we can *see* that there is a difference between the foods and the utilities, but what *exactly* does that mean? 

Outwardly, what we can see is that the *distance* between the 4 food data points to each other is smaller than their distance to the 2 utility points - they are tightly clumped. At the same time, the same is true for the 2 utility data points - though not as clumped, they are closer to each other than to the foods.  

Keep this correlation in mind as we move on to the next section, where we will see how this relationship between the distance of points is related to k-means clustering. 

# 2. Introduction to K-means Clustering

### Basic Theory
By now you should be more familiar with the concept of **clustering**. Let's refer back to the questions that we sought to answer by the end the tutorial:
1. How many clusters are in my data?
2. How do I determine what goes into each cluster?
3. Which clustering of my data is **"most correct"**?  

We briefly saw how *Question 1* depends on both the data that we have, as well as which angle we're looking at. Namely, we saw that when we plotted our `random_objects` data with respect to `log(avg_price)` over `log(avg_mass)` that we could differentiate 2 clusters, a cluster of 'foods' from a cluster of 'utilities'. However, comparing across a different set of *features* could have given us a completely different number of clusters in our final scatterplot representation. For example, throwing `color` into the mix might have given us a situation where we ended up with *3* unique clusters instead of *2*, since our dataset contained items of 3 different colors, `purple`, `yellow`, and `red`.

We also saw how *Question 2* can be answered with our `random_objects` data. At the end of the last section, we observed that the distance between neighboring data points was *inversely proportional* to the strength of their clustering - similar objects should, graphically, be located near each other, because this indicates that their *features* (ex. size, weight, etc.) within a given *feature space* are numerically similar. Thus the *content* of a cluster was given by those points whose *distance between themselves* was smaller than their *distance to the other points in the graph*. 

The question that we've left unanswered, *Question 3*, brings together the two questions before it. It is finally time to introduce ***k-means clustering***.

---
K-means clustering is a method that clusters data into ***k*** number of clusters by trying to *minimize* the *sum of the squared distance* of the data points to the **means** of these clusters, hence *k-means*.  

---
The *sum of squared distances* here is a measure of the *variance* of the data points relative to a cluster mean - in essence, it is answering the question, *"how well are the points in this cluster modeled by this cluster mean?"*. An analogous concept is used in *linear regression* - the R^2 value that we are used to using to explain the "goodness of fit" of a linear model is in fact the same statistical technique used here in k-means clustering, one that tries to minimize the error of the *observations* relative to the model.
![lin reg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Total_least_squares.svg/220px-Total_least_squares.svg.png)

We won't delve into the complex math behind these statistics - it is sufficient to understand that by minimizing the sum of the squared distances between the points in a cluster to their respective cluster mean, k-means clustering *minimizes* the "spread" of each cluster.  

### Basic clustering and calculations
It is understandable that this will not be easy to grasp at first. To improve our learning, so let's try to play around with our `random_objects` dataset in order to illustrate what we mean by k-means clustering. With the data that we plotted, let's assume that our `avg_mass` and `avg_price` data are "correctly" partitioned into 2 clusters (k=2). Given our two clusters, let's find the **centroid** of the cluster, i.e. the geometric **mean** of the points in the cluster.  

To jog our memory, let's bring up our data frame `df` that contained all of our measurements about our `random_objects` in our custom feature space. Since our `avg_price` over `avg_mass` plots showed that the data looks best when log10-transformed, let's permanently change the values in our `avg_mass` and `avg_price` columns to their base 10 log using `np.log10()`, as well as changing the corresponding column names to `log(avg_mass)` and `log(avg_price)`. Changing the values in a data frame is as easy as changing any other object assignment - simply set the new values equal to the old, and Python will store the new values in the dataframe. Once you've finished your code, call `df` again to check your results.
```python
df
df['avg_mass'] = np.log10(df['avg_mass'])
df['avg_price'] = np.log10(df['avg_price'])

## Below, change the 'df.columns' to have `log(avg_mass)` and `log(avg_price)` instead of 'avg_mass' and 'avg_price'
df.columns = ['color','taste','type','log(avg_price)','log(avg_mass)','edible']
```
Good. Let's remember that given our choice to cluster across `avg_mass` and `avg_price`, the two resulting clusters contained the following objects:
* `food_cluster = ['eggplant','grape','banana','plum']`
* `util_cluster = ['school bus', 'fire hydrant']`

We can extract values from our `df` dataframe for these specific clusters by inputing them as arguments to the data frame using the `.loc()` function, like so:
```python
df.loc[food_cluster]
df.loc[util_cluster]
```
We can take this a step further - let's make miniature dataframes from our `df` data frame so that we have two dataframes, `f_cluster` and `u_cluster`, with the `log(avg_mass)` and `log(avg_price)` data of the 'food' and 'utility' objects, respectively. Building on the code above, see if you can define `f_cluster` and `u_cluster` using the `df.loc()` function, and a new list named `features` that contains the features that we're looking at.
```python
features = ['log(avg_mass)','log(avg_price)']
f_cluster = df.loc[food_cluster, features]
u_cluster = df.loc[util_cluster, features]
```
Simple as that.  

A cool thing that we can do with dataframes is use the `.describe()` function to learn more about the columns in the data. Try it out below on both the `f_cluster` and `u_cluster` dataframes. 
```python
f_cluster.describe()
u_cluster.describe()
```
Note all the useful information we get back about our datasets, including the `mean` of each column in the dataframe as well as its standard deviation, `std`. If we want to return an array with a specific value from the `describe()` function, such as the mean, `mean`, we can simply type:
```python
f_cluster.mean()
```
Let's save our two cluster means as new lists called `f_center` and `u_center` - this will give us the coordinates of the center of each of the 'food' and 'utility' clusters that we can add to our earlier plots.
```python
f_center = list(f_cluster.mean())
u_center = list(u_cluster.mean())
```
To check that everything we'd done so far makes sense graphically, let's plot our centers onto the scatterplots that we created earlier. We can recycle all the code that we had previously and simply add the new points to the main plot, like so:
```python
## Initialize the dimensions and axes labels of the 'plot' object
fig, plot = plt.subplots(1, 1)
plot.scatter(df['log(avg_mass)'], df['log(avg_price)'])
plot.set_xlabel('log(avg_mass)')
plot.set_ylabel('log(avg_price)')

## Add our cluster centers, where the '*' breaks up an [x, y] coordinate into a separate x and y value for plotting
plot.plot(*f_center,'ro')
plot.annotate('food center',f_center, rotation=45)

plot.plot(*u_center,'ro')
plot.annotate('utility center',u_center, rotation=45)
```
![Cluster centers](http://i.imgur.com/jwmARuI.png)

Good, now let's stop and think for a bit.  

### 'Within-cluster sum of squares' models
At the beginning of this section, we defined *k-means clustering* as a clustering method that seeks to minimize the sum of squared distances between the centers of the clusters and their associated points. Here in our example, we've plotted exactly that - a representation of our data with k=2 clusters, where the cluster center is the **centroid**, or geometric mean, of the nearby surrounding points. How do we *know* that the centroid minimizes the *within-cluster sum of square distances*? Let's refresh our memory as to the distance formula with the help of our old friend, Pythagoras.
![Pythagorean](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Pythagoras-proof-anim.svg/700px-Pythagoras-proof-anim.svg.png)  
Between any two points in 2-dimensional space, we can calculate the distance between them using *Pythagoras' theorem*. In a cartesian plane, the equation becomes:
![distance formula](http://imgur.com/yrZUa5h)
Where *(x1,y1)* and *(x2,y2)* are the two points that you're comparing. We can easily program this into our code in order to verify that the centroid of our clusters is in fact the point that minimizes the within-cluster sum of squared distances.

Let's begin by initializing two new lists, `food_distances` and `util_distances`, corresponding to our 'food' and 'utility' clusters. To start off, let's work with our `f_cluster` data only - we will come back later and apply our code to `u_cluster` as well. In the space below, write a `for` loop that will take each point in `f_cluster`, find the distance between that point and the `f_center` point we defined earlier, and add that value to the `food_distances` list. (Hint: Use the `np.sqrt()` function, and remember to convert `f_cluster` into an *array* before iterating over it)
```python
food_distances = []
for x,y in np.array(f_cluster):
    # print(x,y) - Add this line of code if you're having issues and want to troubleshoot 
    distance = np.sqrt((x-f_center[0])**2 + (y-f_center[1])**2)
    food_distances.append(distance)
```
Good. Now let's sum up the elements of `food_distances` into a new list, `f_sum`. Since k-means clustering, by defintion, operates to minimize the within-cluster *sum* of square distances, let's make that list by summing the square of each element of `food_distances`. Remember, this *sum of squared distances* measures the *variance* of a set of observations, meaning how spread out the observations are from their mean, or in this case how spread out our data points are from their assigned cluster center.
```python
f_sum = sum(food_distances)
f_sumsq = sum([num**2 for num in food_distances])
```
Good. The same exact code can be used to get the within-cluster sum of *square* (WCSS) distances for the 'utility' cluster, `u_cluster`, so we've done it for you below.
```python
util_distances = []
for x,y in np.array(u_cluster):
    # print(x,y) - Add this line of code if you're having issues and want to troubleshoot 
    distance = np.sqrt((x-u_center[0])**2 + (y-u_center[1])**2)
    util_distances.append(distance)
u_sum = sum(util_distances)
u_sumsq = sum([num**2 for num in util_distances])
```
We can take a look at these values... but at the moment, they won't really tell us much. The easiest way to prove our theory that the centroid of a cluster is the point at which the sum of squared cluster point distances is *minimized* is by purposely "tampering" with our data. We're going to do this in 3 ways:
1. We're going to move the `banana` data point into the utility cluster and see what happens to our WCSS
2. We're going to add a new data point to our data, determine which cluster it belongs to, and see how the WCSS changes
3. Lastly, we're going to cluster the food points around a bogus cluster center, and see how the WCSS is different than when the cluster center is the centroid of the food points

#### Part 1: Assigning a point to the wrong cluster
For Part 1 of this experiment, we've built some new data structures for you to streamline your code:
* `purple_cluster = ['eggplant','grape','plum']`
* `weird_cluster = ['school bus','fire hydrant','banana']`

Keep in mind the following structures and functions that will help you in your task:
* `features = ['log(avg_mass)','log(avg_price)']`
* `<dataframe name>.mean()` - returns the mean of all columns in the dataframe
* `<dataframe name>.loc()` - used to index a dataframe to find the data you want

Your goal for Part 1 is to find the final values of WCSS (which you can name how you wish) for the two new clusters `purple_cluster` and `weird_cluster`, then compare them to our previous values of `u_sum` and `f_sum`. Note that you can reuse any (or all) of the code we've used in previous sections, just make sure you plug in the correct variables.
```python
features = ['log(avg_mass)','log(avg_price)']
p_cluster = df.loc[purple_cluster, features]
w_cluster = df.loc[weird_cluster, features]
p_center = list(p_cluster.mean())
w_center = list(w_cluster.mean())

purple_distances = []
weird_distances = []
for x,y in np.array(p_cluster):
    print(x,y)
    distance = np.sqrt((x-p_center[0])**2 + (y-p_center[1])**2)
    purple_distances.append(distance)
    
for x,y in np.array(w_cluster):
    print(x,y)
    distance = np.sqrt((x-w_center[0])**2 + (y-w_center[1])**2)
    weird_distances.append(distance)

p_sum = sum(purple_distances)
w_sum = sum(weird_distances)

p_sumsq = sum([num**2 for num in purple_distances])
w_sumsq = sum([num**2 for num in weird_distances])
```
Great! For your convenience, we've plotted the new clusters alongside our old ones to see how the two clustering assignments compare.
![cluster vs cluster](http://imgur.com/n5cuK4L)  

As suggested by the name, the 'weird' cluster that we created heavily distorts the previous 'utility' cluster through the inclusion of the `banana` point. Interestingly, the 'purple' cluster hasn't moved much compared to the 'fruit' cluster from which it was derived and with which it shares most of its random objects.  

Let's look at the numbers behind the graph, namely the *within-cluster sum of squares* (WCSS), to see what they say about the "goodness" of our clusters. Remember that a *smaller* WCSS means better clustering, and that the overarching goal of k-means clustering is to minimize WCSS. To make things clearer, we'll also temporarily rename the 'purple' and 'weird' clusters to more straightforward names, namely 'fruit - banana' and 'utility + banana', to get a better feel of how skewing one data point impacted our clusters. 
* WCSS(food): `0.403`
* WCSS(utility): `1.000`  

Sum of WCSS from 1st model: **1.403**
* WCSS(food - banana): `0.256`
* WCSS(utility + banana): `8.720`  

Sum of WCSS from 2nd model: **10.976**  

We can derive 3 main interferences from this data:
1. Interestingly, the previous *'food' cluster seems to cluster better without the 'banana' datapoint*, because its WCSS goes down
2. As expected, the *WCSS for the 'utility + banana' cluster rises significantly* because the 'banana' datapoint is far divorced from 'school bus' and 'fire hydrant' both in terms of log(avg_mass) *and* log(avg_price)
3. Overall, the *1st model is vastly superior to the 2nd model* in terms of clustering as it produces the lower total WCSS, indicating that on average the clusters are more tightly packed

#### Part 2: Figuring out which cluster a new point belongs to
Let's proceed to Part 2 of our experiment - seeing what happens to our within-cluster sum of squares when we add a new datapoint to our original dataframe `df`. The data vector that we've added is called `holiday_ham` and looks like this:
* `holiday_ham` = ['brown','savory','food',35,10, True]  

(Note that here the values of `avg_mass`, 35, and `avg_price`, 10, have not yet been log-transformed)   

When plotted, see how `holiday_ham` lies ambiguously between the 4 'food' data points and 2 'utility' data points that we've been working with.
![Ham included](http://imgur.com/ikpDVQy)

What we want to know is how the *total* WCSS is affected when:
1. Adding `holiday_ham` to the `food` cluster
2. Adding `holiday_ham` to the `utility` cluster 

Note: It is **important** to remember that as soon we add `holiday_ham` to either cluster, we change the cluster mean for that cluster, and within-cluster sum of square distances are calculated from the *new* cluster center, rather than the old.

Since you're already familiar with how to implement this part of the analyis, we'll only go through the *basic* steps required to do it before jumping straight to the final analysis. The steps that you should be familiar with by now are: 
- Adding to/constructing a dataframe using `pd.DataFrame` from the *pandas* library
- Creating "mini" dataframes by slicing/indexing using `pd.loc()`
- Checking statistics about your data using `pd.describe()`, and pulling out values such as `std` (standard deviation) and `mean`
- Calculating sum of square distances between cluster means and their points using `for` loops and Pythagoras' theorem  

We've already calculated the new WCSS values for you below. Note that we can reuse the within-cluster sum of squares from both our `utility` and `food` clusters from before, when needed, as long as the clusters haven't changed identity.  

* WCSS(food + ham): `2.111`
* WCSS(utility): `1.000` (reused from before) 

Sum of WCSS from including `holiday ham` with the 'food' cluster: **3.111**

* WCSS(utility + ham): `4.326`
* WCSS(food): `0.403` (reused from before) 

Sum of WCSS from including `holiday_ham` with the 'utility' cluster: **4.729**  

Based on the WCSS, we've shown that `holiday_ham` does in fact cluster better with the 'food' group than with the 'utility' group because *that* clustering model produces a *lower overall within-cluster sum of squared distances*, thus tighter packing of clusters compared to lumping `holiday_ham` together with the 'utility' group. Once again, remember that by adding `holiday_ham` to the `food` cluster, we have redefined the cluster mean, its contents, and its WCSS compared to the original.  

#### Part 3: Using a bad cluster center
For the last part of our experiment, we've defined a fake cluster center for the 'food group' (not including the `holiday_ham` that we added in the previous part) called `bad_center` located at [0, 1.5] (values *already* log-transformed), such that the cluster has shifted as seen in the figure below:
![bogus food center](http://imgur.com/edn9fEZ)  

It is clear that is not the proper center of the food data, but let's prove it mathematically by once again looking at the within-cluster sum of squares. Our WCSS(utility) value should not change, because neither the 'utility' cluster nor its contents have changed, but since the 'food' datapoints are disjoint and scattered away from the `bad_center`, we predict that the WCSS around this new cluster center will be higher, thus representing a worse clustering model. Let's take a look at the values to prove it:

* WCSS(food, original): `0.403` (reused from before)
* WCSS(utility): `1.000` (reused from before)

Sum of WCSS using the *centroid* as the cluster center: **1.403**

* WCSS(food, bad): `4.486`
* WCSS(utility): `1.000` (reused from before)

Sum of WCSS using a bogus 'food' cluster center: **5.486**  

As predicted, the use of an improper cluster center had a profound negative effect on the clustering model. Now that we've seen how perturbations or new additions to our dataset affect our WCSS, let's look into how exactly k-means clustering seeks to create the *best* clustering model given a dataset.

### The k-means algorithm
What we've set up to do with the previous two sections is demonstrate the *spirit* of k-means clustering - it is a clustering method that seeks to minimize the within-cluster sum of squared distances, thereby reducing the spread of the k number of clusters that you're trying to pull out of your data.  

The k-means clustering ***algorithm***, as in the computational steps that are taken to find the best k number of cluster centers, proceeds through a number of *iterative* steps, that are very similar to the 3-part experiment that we conducted in the previous section.  
* **First step**: K number of *random points* are chosen as the cluster centers. These points can be terrible choices for cluster means, as we saw in Part 3 with `bad_center`, but it nonetheless gives us a starting point with which to work with
* **Second step**: The nearest points to these randomly-assigned cluster centers are assigned to those clusters by minimizing WCSS. This is exactly what we saw in Part 2 with the addition of `holiday_ham` - we figured out it belongs to 'food' cluster instead of the 'utility' cluster because WCSS(food + ham) was *lower* than WCSS(utility + ham)
* **Third step**: The cluster center is *redetermined*, based on the new points assigned to it - the *centroid* changes. We saw this happen in *both* Part 2 and Part 1 - adding `holiday_ham` to the 'food' cluster changed the cluster mean, and so too did redesignating `banana` as a member of the 'utility' cluster instead of the 'food' cluster
* Finally, **repeat Steps 2 and 3** until you reach an equilibrium where the *cluster centers no longer move*  

An easy way to understand how the k-means algorithm operates is by watching an example of it in real time. Note in the example below that *random* data is being used, *N=200* indicates that there are 200 random data points in the dataset, *K=3* refers to have many clusters we're looking for, and the number of *iterations* indicates how many "cycles" of Steps 2 and 3 in the list above it took before the cluster centers (stars) stopped changing.
![Algorithm demonstration 2](https://datasciencelab.files.wordpress.com/2013/12/p_n200_k3.gif)  

As stated before, note how the k-means algorithm is *iterative* in the sense that each step, each movement of the cluster centers, begins from the previous coordinates of the cluster center and brings the center closer and closer to the *optimal* solution of the clustering problem, the solution that *minimizes the total sum of within-cluster sum of squared distances*. For those of you who are more mathematically-inclined, the k-means algorithm can be expressed as finding the minimum of the following equation:
![k means equation](https://s0.wp.com/latex.php?latex=%5Cdisplaystyle+%5Csum_%7Bk%3D1%7D%5EK+%5Csum_%7B%5Cmathrm%7Bx%7D_n+%5Cin+C_k%7D+%7C%7C%5Cmathrm%7Bx%7D_n+-+%5Cmu_k+%7C%7C%5E2&bg=ffffff&fg=000000&s=1)  

Where K is the desired number of clusters, x-sub-n are the n-number of observations that you have in your dataset (`random_objects`, in our previous examples), and u-sub-k are the K number of cluster centers. The capital greek letter sigma here signifies a 'sum'; we are first calculating the within-cluster sum of squares per cluster, and then summing up the *total* within-cluster sum of squares of all clusters as the quantity that we're trying to *minimize* in order to reach the best clustering model.  

#### Limitations
We've thrown the word *'best'* around a lot while describing k-means clustering, but it is fair to realize some limitations of the technique. As we said earlier, the within-cluster sum of squares that the k-means algorithm seeks to minimize is analogous to the R^2 measure of correlation that we use for linear regression, but consider the following example taken from "Anscombe's Quartet":
![Anscombe](http://varianceexplained.org/figs/2015-01-16-kmeans-free-lunch/anscombe-1.png)  

Each of the 4 datasets in Anscombe's Quartet **has the same value of R^2**, yet clearly the linear model superimposed on each of the sets of data is intuitively *not* the best line of fit.  

K-means clustering can fall into similar pitfalls when your dataset has outliers, or when the clusters in your dataset don't have a uniform, *globular* shape. For instance, take this example of k-means clustering gone wrong:
![incorrect kmeans](http://varianceexplained.org/figs/2015-01-16-kmeans-free-lunch/plot_kmeans-1.png)  

In the original dataset, it was apparent that the "outer" circle and "inner" circle consistituted two separate clusters of observations, yet the k-means algorithm naturally lends itself to finding *globular*, or spherical clusters as a solution to finding the minimum WCSS. Indeed, multiple clustering algorithms exist that have different criteria for defining the *"goodness"* of a cluster model, beyond the minimum WCSS criteria of k-means. Returning to the example above, *hierarchical clustering*, a clustering method that, broadly, creates clusters based on the interconnectivity of points in a dataset rather than using pre-defined cluster means, *does* partition the above data correctly.  To that end, it is important as data scientists to anticipate which algorithm will best suit your dataset.  

Now that you understand how the k-means algorithm works, let's wrap up our discussion with a demonstration of what can be done with some open-source Python libraries designed *specifically* for k-means analysis, while also raising the stakes and working with a larger, real data set.

### sklearn and the 'iris' dataset
For k-means clustering, multiple modules are available that streamline the analysis, the most popular of which comes from the `scikit-learn` library. More info about this library can be found [at this link](http://scikit-learn.org/stable/index.html) - it is a powerful resource for any data scientist using Python, and we heavily encourage you to check it out. The module that we'll be importing is called `sklearn.cluster`, and from it will import the `KMeans` class, like so:

```python
from sklearn.cluster import KMeans
```
You can learn more about the `KMeans` class and its functions on the 'scikit learn' website, or by typing `?KMeans` after importing it.

For the rest of the lesson, we will be using the **iris** dataset, an open-source dataset also available from the `scikit-learn` library that was introduced in the early 20th century by Ronald Fisher and contains morphological data about 3 types of iris flowers (50 samples from each type). Let's load it in to have a closer look. We can do so by executing the following code and saving the dataset as the variable `iris`:
```python
from sklearn.datasets import load_iris
iris = load_iris()
```
Typing `iris` into your Python console, you can see that there is a plethora of information about the dataset. Let's focus on a few specific attributes of the dataset:
* The **'data'** array that contains the raw data values  
* The **'feature_names'** array containing the *features*, equivalent to our dataframe *columns*
* The **'target'** array which contains the integer designation of the identity of each sample, equivalent to our dataframe *index*  
* And the **'target_names'** arrays which contains the "translation" of the integer identities in 'target' to their corresponding iris flower

For your convenience, we're going to convert the `iris` data to its corresponding dataframe, `iris_df`, for you. Remember the 3 attributes necessary to define a dataframe: 1) the data, 2) the indices (row names), and 3) the columns (column names). Make sure that you can follow the code. (Note that in our code, we won't be giving our dataframe *named* indices the same way we did with our `random_objects` list because in the `iris` dataset there would be 50 repeats of each index name, corresponding to the 50 samples of each iris flower type). 
```python
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
```
Our handy `iris_df.columns` attribute tells us that our dataset is looking at the following features:
- Sepal length (cm)
- Sepal width (cm)
- Petal length (cm)
- Petal width (cm)
- Target

Treating this dataset just like our `random_objects` dataset from before, let's try to run an exploratory scatterplot using the `sepal length (cm)` and `sepal width (cm)` data to see if we can make any predictions or observations about any inherent clusters in the data. In the space below, make a scatterplot of `sepal width (cm)` over `sepal length (cm)` across all the `iris` datapoints using the slicing methods we've used before, and remember that since we don't have explicit row names such as `eggplant` this time around, you'll have to index rows *by number* in future parts of this section.

```python
plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')

### Your code here
plt.scatter(iris_df['sepal length (cm)'], iris_df['sepal width (cm)'])
```
![raw iris](http://imgur.com/TSZYj6A)  

Good. Note how we didn't have to log-transform the data in order to get good separation of our datapoints with the `iris` dataset. This dataset is also visibly more dense than our previous `random_objects` dataset. This means that our choice of cluster means and the overall complexity of the solution to the k-means algorithm for this dataset will be more complex.  

To improve our sense of potential clusters in the data, let's embellish our plot by coloring in each datapoint in a different color depending on the `target` column of the `iris_df` dataframe, corresponding to which 'species' the observation belongs to. Let's do this by plotting 3 plots, 1 for each type of iris flower, on the same axes. We can do this using boolean logic slicing of our `iris_df`. For instance, the following code...
```python
iris_df.loc[iris_df['target']==0]
```
... Essentially translates to *"from `iris_df`, give me all the data for where `target` is equal to 0"*. As usual, we will define our `features` as the feature space that we're interested in:
```python
features = ['sepal width (cm)', 'sepal length (cm)']
```
Given these tools, in the space below create a scatterplot from the `iris_df` data such that each point is colored based on the iris flower species it was gathered from. To improve the quality of our plot, add a plot legend using `plt.legend()`, with each plot label being derived from the following mapping between `target` and flower name:
- Setosa = 0, Versicolor = 1, Virginica= 2  

```python
### Data acquisition
c1 = iris_df.loc[iris_df['target']==0, features]
c2 = iris_df.loc[iris_df['target']==1, features]
c3 = iris_df.loc[iris_df['target']==2, features]

### Plotting
fig, plot1 = plt.subplots()
plot1.scatter(c1['sepal length (cm)'], c1['sepal width (cm)'], c='r', label='setosa')
plot1.scatter(c2['sepal length (cm)'], c2['sepal width (cm)'], c='y', label='versicolor')
plot1.scatter(c3['sepal length (cm)'], c3['sepal width (cm)'], c='m', label='virginica')
plot1.set_xlabel('sepal length (cm)')
plot1.set_ylabel('sepal width (cm)')
plot1.legend()
```
![iris colored](http://imgur.com/izZoFpx)  

Great job. With the points colored by their respective flower, we can see that the `setosa` datapoints are clearly separated from the `versicolor` and `virginica` datapoints with respect to both sepal width and length. The latter two datasets, however, have a lot of overlap - it will be interesting to see how the k-means algorithm segregates these datapoints. The advantage of using the `iris` dataset for this tutorial is that we come equipped with the prior knowledge that our dataset contains *3 distinct types of flower* - in clustering terms, the "k" number of clusters is essentially already decided to be 3. However, what does it mean if the clusters produced by k-means clustering do not line up perfectly with the observations that we've plotted above for `sepal width (cm)` and `sepal length (cm)`? 

#### Clustering using sklearn.cluster.KMeans
To answer this last question let's perform a standard KMeans analysis of our `iris` data with the `KMeans` class. A detailed description of this class, including the different types of arguments that it can take and their default parameters, is available at [the following link](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans). For now, we will be using the `.fit()` function of `KMeans` in order to cluster our data into a new `KMeans` object named `iris_result`. For this demonstration we will provide and walk you through the code, but we heavily encourage you to play around with changing the arguments, such the number of final clusters, after this lesson.
```python
iris_result = KMeans(n_clusters=3).fit(iris_df[features])
```
When you've successfully run this code, you can run `iris_result` in your console to see some additional information about how the k-means analysis was performed, the 4 important ones being:
* **`init='k-means++'`**: Specifies the method used to determine the *initial* cluster centers in the data, around which the iterative k-means algorithm begins. 'k-means++' is one such method, and is generally better than choosing random points for the initial cluster centers
* **`max_iter=300`**: Specifies how many iterations of the k-means algorithm will be attempted before the function essentially "gives up", having not settled on a clustering solution. 
* **`n_clusters=3`**: Specifies how many clusters we want to pull out of our data.
* **`n_init=10`**: Specifies how many different initial cluster configurations we want to test - the `KMeans.fit()` function will then use the one that yields the lowest WCSS.  

As you can see both in your output and in the online documentation, the `KMeans.fit()` function has many variations that you can implement to suit your data and improve your clustering analysis. (We won't go into the different types of initialization strategies that can be used, such as 'k-means++', because our data is simple enough that it won't change our end result, but please feel free to explore them in your own data analysis projects).  

Our new `iris_result` object comes with numerous useful attributes including:
* **`.cluster_centers_`**: Coordinates of the final cluster centers
* **`.labels_`**: Labels of each points, i.e. which of the k clusters each point was assigned to
* **`.inertia_`**: Another way of saying *total within-cluster sum of squares*  

To bring our analysis of k-means full circle, let's visualize how the `KMeans.fit()` function clustered our data and compare it to the intrinsic separation of the data points in the `sepal width (cm)` and `sepal length (cm)` feature space that we plotted above. While we will provide the necessary code, we encourage you to try doing it yourself, following the previous plot as a template for your code. (Hint: A useful first step is to add the `iris_result.labels_` column to the `iris_df` dataframe as, so that you can slice the dataset according to the labels produced by the k-means analysis).
```python
iris_df['cluster label']=iris_result.labels_

km1 = iris_df.loc[iris_df['cluster label']==0, features]
km2 = iris_df.loc[iris_df['cluster label']==1, features]
km3 = iris_df.loc[iris_df['cluster label']==2, features]

fig, plt1 = plt.subplots()
plt1.scatter(km1['sepal length (cm)'], km1['sepal width (cm)'], c='y', label='cluster 0')
plt1.scatter(km2['sepal length (cm)'], km2['sepal width (cm)'], c='m', label='cluster 1')
plt1.scatter(km3['sepal length (cm)'], km3['sepal width (cm)'], c='r', label='cluster 2')
plt1.set_xlabel('sepal length (cm)')
plt1.set_ylabel('sepal width (cm)')
plt1.legend()
```
![sklearn clusters](http://imgur.com/2PDwd2i)
Fantastic. Let's recap what steps we've taken and what this graph shows before concluding this lesson. The steps that this k-means function took were:
1. Defining 3 cluster centers using the 'k-means++' method
2. Assigning points to those clusters based on their distance to the cluster center - a step that minimizes WCSS
3. Redefining the cluster centers so that they're the *centroid*, or geometric mean, of their substituent points - another step that minimizes WCSS
4. Repeat until the cluster centers don't change *OR* we hit the `max_iter` value  
By minimizing the within-cluster sum of squares at each step, this k-means functions creates clusters in areas of the data that are "tightly packed". For instance, compare the `setosa` data from the original dataset to the `cluster 2` data from the k-means-clustered dataset - except for 1 outlier, the two sets of datapoints are identical. This agreement between the two datasets suggest that the original `setosa` data, in regards to the given feature space, is naturally well-clustered and has a unique identity in regards to `sepal width (cm)` and `sepal length (cm)` when compared to the more homogenous `versicolor` and `virginica` data.   

The fact that this k-means clustering operation could not *perfectly* tease apart all the `versicolor` datapoints from the `virginica` is a commentary on both the nature of the algorithm *and* of the dataset - `versicolor` and `virginica` are intrinsically similar to each other (, or both data were filled with many outliers,) in the given feature space, which is a natural limitation of the k-means algorithm in that it naturally finds globular clusters and is sensitive to outliers. The k-means algorithm did not *fail* in this case, but this illustrates how alternate clustering techniques, or using different weightings of features (ex. using the *square* of `sepal width (cm)**` to model the data), are sometimes a better choice.

# 3. Concluding remarks
Let us briefly review the concepts that we've covered:
* Clustering aims to lump together objects by virtue of some property of their *features*   
* K-means clustering is one of many algorithms that define how *good* a particular clustering model is
    * The process of k-means clustering is *iterative*, built on three main phases that aim to minimize the within-cluster sum of squares
        * **Initiation**: Initial cluster clusters are assigned through various methods
        * **Build clusters**: Given initial clusters, assign data points to their respective cluster by minimizing distance
        * **Redefine centers**: Update the cluster means, or *centroids*, based on the points assigned to them
        * Repeat the previous two steps until there is no change in cluster centers
    * K-means has a few limitations
        * Sensitive to outliers
        * Produces uniform, globular clusters which don't always do the data justice  
        
With the exercises and solutions presented here, I hope that you now have a firm hold on the basic principles behind k-means clustering, as well as permutations thereof. I also hope that you are now more familiar with some aspects of the `sklearn` library, as well as `pandas` dataframes and some of the plotting techniques available through `matplotlib`.

To reiterate, k-means clustering is but one algorithm in the family of clustering, but it is a widespread and fundamental tool in any data analyst's skillset. If you are interested in learning more about k-means clustering as well as other available datasets, we highly recommend visiting the scikit-learn website at http://scikit-learn.org/stable/ for additional resources.